# Notes
- If meet id inclues a `/`, it is probably wrong.  Still don't know why, but we won't use Meet_ID anyways.
- Kyle has TWO athlete id's, 7109122 and 6459186.  Could be a problem.

In [128]:
import pandas as pd
import numpy as np
import utils
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)

In [129]:
mdf = pd.read_csv('M_athlete_results.csv')
fdf = pd.read_csv('F_athlete_results.csv')

C:\ProgramData\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [130]:
# String marks to float times in seconds
mdf['Time'] = [utils.Time(m) if (e in utils.event_classification()['Track'] and utils.ismark(m)) else None for e, m in zip(mdf['Event'], mdf['Mark'])]
fdf['Time'] = [utils.Time(m) if (e in utils.event_classification()['Track'] and utils.ismark(m)) else None for e, m in zip(fdf['Event'], fdf['Mark'])]

In [131]:
# String events to float distance in meters
mdf['Distance'] = [utils.string_to_distance(e) if e in utils.event_classification()['Track'] else None for e in mdf['Event']]
fdf['Distance'] = [utils.string_to_distance(e) if e in utils.event_classification()['Track'] else None for e in fdf['Event']]

In [132]:
csm_m = mdf.loc[(mdf['School'] == 'COLORADO MINES') & (mdf['Season'] == 'Outdoor') & (mdf['Event'] == '5000')]
csm_f = fdf.loc[(fdf['School'] == 'COLORADO MINES') & (fdf['Season'] == 'Outdoor') & (fdf['Event'] == '5000')]

In [133]:
xcnats_m = mdf.loc[mdf['Meet_Name'] == 'NCAA Division II Cross Country Championships'].sort_values(['Place', 'Mark'])
xcnats_f = fdf.loc[fdf['Meet_Name'] == 'NCAA Division II Cross Country Championships'].sort_values(['Place', 'Mark'])

In [134]:
f5 = fdf.loc[(fdf['Season'] == 'Outdoor') & (fdf['Distance'] == 5000.0)]
m5 = mdf.loc[(mdf['Season'] == 'Outdoor') & (mdf['Distance'] == 5000.0)]